# Definir eventos locales

In [13]:
# Tabla de eventos extremos para análisis en GEE/Colab
# Cada evento tiene un área aproximada definida por 3-5 puntos, fechas en formato GEE,
# variables principales con ponderación lógica, y descripción del fenómeno.

events_area = [
    {
        "name": "Sequía extrema",
        "region": "California, EE. UU.",
        "coords": [  # área aproximada (polígono)
            [36.8, -120.0],
            [36.8, -119.0],
            [35.9, -119.0],
            [35.9, -120.0]
        ],
        "start_date": "2014-01-01",
        "end_date": "2017-12-31",
        "variables": {  # ponderación de impacto en el índice de estrés
            "NDVI": 0.5,
            "precipitation": 0.4,
            "evapotranspiration": 0.1
        },
        "description": "Sequía prolongada afectando agricultura y provocando incendios forestales."
    },
    {
        "name": "Calor extremo",
        "region": "Australia (Sur y Este)",
        "coords": [
            [-34.0, 150.5],
            [-34.0, 152.0],
            [-32.5, 152.0],
            [-32.5, 150.5]
        ],
        "start_date": "2019-01-01",
        "end_date": "2020-12-31",
        "variables": {
            "T2M": 0.7,       # temperatura diurna principal
            "NDVI": 0.2,      # afectación indirecta a vegetación
            "ET": 0.1         # pérdida de agua
        },
        "description": "Ola de calor récord afectando estrés en cultivos."
    },
    {
        "name": "Plagas agrícolas",
        "region": "India (Punjab)",
        "coords": [
            [30.5, 75.5],
            [30.5, 76.2],
            [31.3, 76.2],
            [31.3, 75.5]
        ],
        "start_date": "2020-06-01",
        "end_date": "2020-09-30",
        "variables": {
            "NDVI": 0.6,      # vegetación afectada
            "T2M": 0.3        # temperatura favorece plagas
        },
        "description": "Brotes de langostas del desierto afectando trigo y otros cultivos."
    },
    {
        "name": "Precipitaciones fluviales",
        "region": "Bangladesh (Delta del Ganges)",
        "coords": [
            [23.5, 90.0],
            [23.5, 91.2],
            [24.2, 91.2],
            [24.2, 90.0]
        ],
        "start_date": "2017-07-01",
        "end_date": "2017-09-30",
        "variables": {
            "precipitation": 0.7,
            "soil_moisture": 0.3
        },
        "description": "Inundaciones por lluvias monzónicas afectando arrozales."
    },
    {
        "name": "Heladas / Frío extremo",
        "region": "Rusia (Siberia)",
        "coords": [
            [61.0, 104.0],
            [61.0, 107.0],
            [62.5, 107.0],
            [62.5, 104.0]
        ],
        "start_date": "2018-01-01",
        "end_date": "2018-03-31",
        "variables": {
            "Tmin": 0.6,   # temperatura mínima afecta directamente
            "NDVI": 0.4    # vegetación afectada
        },
        "description": "Heladas severas afectando cultivos de cereales y hortalizas."
    },
    {
        "name": "F. de El Niño / Lluvias intensas",
        "region": "Perú (Costa norte y central)",
        "coords": [
            [-5.2, -81.0],
            [-5.2, -80.3],
            [-12.2, -77.0],
            [-12.2, -76.5]
        ],
        "start_date": "2017-01-01",
        "end_date": "2017-04-30",
        "variables": {
            "precipitation": 0.6,
            "NDVI": 0.3,
            "soil_moisture": 0.1
        },
        "description": "Lluvias extremas, inundaciones y desbordes de ríos afectando cultivos y suelos."
    }
]

# ✅ Nota:
# - coords → polígono aproximado, no demasiados puntos para simplificar extracción en GEE.
# - variables → ponderación de impacto, suman 1.0, coherente con la relevancia de cada variable para el evento.
# - fechas → formato GEE "YYYY-MM-DD".


In [14]:
import pandas as pd

# Convertir lista de eventos a DataFrame
data = []
for event in events_area:
    data.append({
        "Nombre": event["name"],
        "Región": event["region"],
        "Fecha Inicio": event["start_date"],
        "Fecha Fin": event["end_date"],
        "Variables (ponderación)": ", ".join([f"{k}: {v}" for k,v in event["variables"].items()]),
        "Descripción": event["description"]
    })

df = pd.DataFrame(data)
df


,Nombre,Región,Fecha Inicio,Fecha Fin,Variables (ponderación),Descripción
0,Sequía extrema,"California, EE. UU.",2014-01-01,2017-12-31,"NDVI: 0.5, precipitation: 0.4, evapotranspirat...",Sequía prolongada afectando agricultura y prov...
1,Calor extremo,Australia (Sur y Este),2019-01-01,2020-12-31,"T2M: 0.7, NDVI: 0.2, ET: 0.1",Ola de calor récord afectando estrés en cultivos.
2,Plagas agrícolas,India (Punjab),2020-06-01,2020-09-30,"NDVI: 0.6, T2M: 0.3",Brotes de langostas del desierto afectando tri...
3,Precipitaciones fluviales,Bangladesh (Delta del Ganges),2017-07-01,2017-09-30,"precipitation: 0.7, soil_moisture: 0.3",Inundaciones por lluvias monzónicas afectando ...
4,Heladas / Frío extremo,Rusia (Siberia),2018-01-01,2018-03-31,"Tmin: 0.6, NDVI: 0.4",Heladas severas afectando cultivos de cereales...
5,F. de El Niño / Lluvias intensas,Perú (Costa norte y central),2017-01-01,2017-04-30,"precipitation: 0.6, NDVI: 0.3, soil_moisture: 0.1","Lluvias extremas, inundaciones y desbordes de ..."


In [15]:
import folium

# Crear un mapa base centrado en el mundo
m = folium.Map(location=[0, 0], zoom_start=2)

# Añadir polígonos para cada evento
for event in events_area:
    folium.Polygon(
        locations=event["coords"],
        color="blue",
        fill=True,
        fill_opacity=0.3,
        popup=f"{event['name']} ({event['region']})"
    ).add_to(m)

# Mostrar mapa
m


# Definir forma de mapeo de eventos en mid game

In [25]:
# Tabla de eventos extremos para análisis en GEE/Colab
# Cada evento tiene un área aproximada definida por 3-5 puntos, fechas en formato GEE,
# variables principales con ponderación lógica, y descripción del fenómeno.

events_area = [
    {
        "name": "Sequía extrema",
        "region": "California, EE. UU.",
        "coords": [  # área aproximada (polígono)
            [36.8, -120.0],
            [36.8, -119.0],
            [35.9, -119.0],
            [35.9, -120.0]
        ],
        "start_date": "2014-11-01",
        "end_date": "2015-04-30",
        "variables": {  # ponderación de impacto en el índice de estrés
            "NDVI": 0.5,
            "precipitation": 0.4,
            "evapotranspiration": 0.1
        },
        "description": "Sequía prolongada afectando agricultura y provocando incendios forestales."
    },
    {
        "name": "Calor extremo",
        "region": "Australia (Sur y Este)",
        "coords": [
            [-34.0, 150.5],
            [-34.0, 152.0],
            [-32.5, 152.0],
            [-32.5, 150.5]
        ],
        "start_date": "2019-09-01",
        "end_date": "2020-02-29",
        "variables": {
            "T2M": 0.7,       # temperatura diurna principal
            "NDVI": 0.2,      # afectación indirecta a vegetación
            "ET": 0.1         # pérdida de agua
        },
        "description": "Ola de calor récord afectando estrés en cultivos."
    },
    {
        "name": "Plagas agrícolas",
        "region": "India (Punjab)",
        "coords": [
            [30.5, 75.5],
            [30.5, 76.2],
            [31.3, 76.2],
            [31.3, 75.5]
        ],
        "start_date": "2020-06-01",
        "end_date": "2020-09-30",
        "variables": {
            "NDVI": 0.6,      # vegetación afectada
            "T2M": 0.3        # temperatura favorece plagas
        },
        "description": "Brotes de langostas del desierto afectando trigo y otros cultivos."
    },
    {
        "name": "Precipitaciones fluviales",
        "region": "Bangladesh (Delta del Ganges)",
        "coords": [
            [23.5, 90.0],
            [23.5, 91.2],
            [24.2, 91.2],
            [24.2, 90.0]
        ],
        "start_date": "2017-07-01",
        "end_date": "2017-09-30",
        "variables": {
            "precipitation": 0.7,
            "soil_moisture": 0.3
        },
        "description": "Inundaciones por lluvias monzónicas afectando arrozales."
    },
    {
        "name": "Heladas / Frío extremo",
        "region": "Rusia (Siberia)",
        "coords": [
            [61.0, 104.0],
            [61.0, 107.0],
            [62.5, 107.0],
            [62.5, 104.0]
        ],
        "start_date": "2018-01-01",
        "end_date": "2018-03-31",
        "variables": {
            "Tmin": 0.6,   # temperatura mínima afecta directamente
            "NDVI": 0.4    # vegetación afectada
        },
        "description": "Heladas severas afectando cultivos de cereales y hortalizas."
    },
    {
        "name": "F. de El Niño / Lluvias intensas",
        "region": "Perú (Costa norte y central)",
        "coords": [
            [-5.2, -81.0],
            [-5.2, -80.3],
            [-12.2, -77.0],
            [-12.2, -76.5]
        ],
        "start_date": "2017-01-01",
        "end_date": "2017-04-30",
        "variables": {
            "precipitation": 0.6,
            "NDVI": 0.3,
            "soil_moisture": 0.1
        },
        "description": "Lluvias extremas, inundaciones y desbordes de ríos afectando cultivos y suelos."
    }
]

# ✅ Nota:
# - coords → polígono aproximado, no demasiados puntos para simplificar extracción en GEE.
# - variables → ponderación de impacto, suman 1.0, coherente con la relevancia de cada variable para el evento.
# - fechas → formato GEE "YYYY-MM-DD".


In [26]:
import pandas as pd

# Convertir lista de eventos a DataFrame
data = []
for event in events_area:
    data.append({
        "Nombre": event["name"],
        "Región": event["region"],
        "Fecha Inicio": event["start_date"],
        "Fecha Fin": event["end_date"],
        "Variables (ponderación)": ", ".join([f"{k}: {v}" for k,v in event["variables"].items()]),
        "Descripción": event["description"]
    })

df = pd.DataFrame(data)
df


,Nombre,Región,Fecha Inicio,Fecha Fin,Variables (ponderación),Descripción
0,Sequía extrema,"California, EE. UU.",2014-11-01,2015-04-30,"NDVI: 0.5, precipitation: 0.4, evapotranspirat...",Sequía prolongada afectando agricultura y prov...
1,Calor extremo,Australia (Sur y Este),2019-09-01,2020-02-29,"T2M: 0.7, NDVI: 0.2, ET: 0.1",Ola de calor récord afectando estrés en cultivos.
2,Plagas agrícolas,India (Punjab),2020-06-01,2020-09-30,"NDVI: 0.6, T2M: 0.3",Brotes de langostas del desierto afectando tri...
3,Precipitaciones fluviales,Bangladesh (Delta del Ganges),2017-07-01,2017-09-30,"precipitation: 0.7, soil_moisture: 0.3",Inundaciones por lluvias monzónicas afectando ...
4,Heladas / Frío extremo,Rusia (Siberia),2018-01-01,2018-03-31,"Tmin: 0.6, NDVI: 0.4",Heladas severas afectando cultivos de cereales...
5,F. de El Niño / Lluvias intensas,Perú (Costa norte y central),2017-01-01,2017-04-30,"precipitation: 0.6, NDVI: 0.3, soil_moisture: 0.1","Lluvias extremas, inundaciones y desbordes de ..."


In [27]:
import folium

# Crear un mapa base centrado en el mundo
m = folium.Map(location=[0, 0], zoom_start=2)

# Añadir polígonos para cada evento
for event in events_area:
    folium.Polygon(
        locations=event["coords"],
        color="blue",
        fill=True,
        fill_opacity=0.3,
        popup=f"{event['name']} ({event['region']})"
    ).add_to(m)

# Mostrar mapa
m


In [22]:
ubicaciones_fenomenos = [
    {
        "fenomeno": "Sequía extrema",
        "pais_region": "California, EE. UU.",
        "ubicacion": "Valle Central / zona agrícola cerca de Fresno",
        "coordenadas": [-119.7871, 36.7378],  # punto representativo en Fresno agrícola :contentReference[oaicite:0]{index=0}
        "fecha_inicio": "2014-11-01",
        "fecha_fin": "2015-04-30",
        "descripcion": "Sequía prolongada afectando agricultura e incendios forestales"
    },
    {
        "fenomeno": "Calor extremo",
        "pais_region": "Australia (Sur y Este)",
        "ubicacion": "zona agrícola en Victoria cerca de Melbourne",
        "coordenadas": [145.0, -37.8],  # punto aproximado en zona agrícola del sudeste de Victoria
        "fecha_inicio": "2019-09-01",
        "fecha_fin": "2020-02-29",
        "descripcion": "Ola de calor récord afectando estrés en cultivos"
    },
    {
        "fenomeno": "Plagas agrícolas",
        "pais_region": "India (Punjab)",
        "ubicacion": "zona agrícola en Punjab cerca de Ludhiana",
        "coordenadas": [75.85, 30.90],  # punto representativo en Punjab
        "fecha_inicio": "2020-06-01",
        "fecha_fin": "2020-09-30",
        "descripcion": "Brotes de langostas del desierto afectando trigo y otros cultivos"
    },
    {
        "fenomeno": "Precipitaciones fluviales",
        "pais_region": "Bangladesh (Delta del Ganges)",
        "ubicacion": "zona agrícola en los distritos inundables cerca de Dhaka",
        "coordenadas": [90.5, 23.8],  # punto aproximado en zona agrícola deltaica
        "fecha_inicio": "2017-07-01",
        "fecha_fin": "2017-09-30",
        "descripcion": "Inundaciones por lluvias monzónicas, afectando arrozales"
    },
    {
        "fenomeno": "Heladas / Frío extremo",
        "pais_region": "Rusia (Siberia)",
        "ubicacion": "zona agrícola en Siberia central dentro del polígono",
        "coordenadas": [105.5, 61.8],  # punto estimado dentro del polígono
        "fecha_inicio": "2018-01-01",
        "fecha_fin": "2018-03-31",
        "descripcion": "Heladas severas que afectaron cultivos de cereales y hortalizas"
    },
    {
        "fenomeno": "Fenómeno de El Niño / Lluvias intensas",
        "pais_region": "Perú (Costa norte y central)",
        "ubicacion": "zona agrícola en Piura / costa norte",
        "coordenadas": [-80.63, -5.19],  # punto representativo en Piura agrícola
        "fecha_inicio": "2017-01-01",
        "fecha_fin": "2017-04-30",
        "descripcion": "Lluvias extremas, inundaciones y desbordes de ríos, afectando cultivos y suelos"
    }
]
